# Orr Sommerfeld pseudospectra

Let's use the classic Orr-Sommerfeld problem to explore *pseudospectra* with Dedalus. 

The Orr-Sommerfeld problem explores the stability of flow down a rectangular channel driven by a pressure gradient. The equilibrium solution is known analytically as Plane Poiseuille flow, and is given by 

$$U(z) = 1 - z^2.$$

Orr and Sommerfeld noted that this becomes unstable despite having no inflection point in the flow. This means that the viscosity itself is the source of the instability.

This is a very interesting problem because the linear operator $\mathcal{L}_{OS}$ is not normal, that is $\mathcal{L}_{OS}^\dagger \mathcal{L}_{OS} \neq \mathcal{L}_{OS}\mathcal{L}_{OS}^\dagger$. This means it can have transient growth: even systems in which *all* modes are linearly stable (that is, they will eventually decay) can grow to rather large amplitudes.

Here, we assume perturbations take the form $u(x,z,t) = \hat{u} e^{i\alpha (x - ct)}$, where $c = \omega/\alpha$ is the wave speed. One must take care to note that $c = c_r + i c_i$. If a perturbation has $c_i > 0$, it will linearly grow. This occurs at a Reynolds number $\mathrm{Re} \simeq 5775$, a result you can also find using `eigentools` (it's actually a test problem).

However, Reddy, Schmid, and Henningson (1993) noted that even at $\mathrm{Re} = 10000$, the growth rate is tiny: $c_i \approx 3.7\times10^{-3}$! Yet at those Reynolds numbers, channel flow is very turbulent. How can this be?

The answer is that the *spectrum* of the Orr-Sommerfeld operator is quite misleading. Because $\mathcal{L}_{OS}$ is non-normal, its eigenfuntions are not orthogonal. As they decay at different rates, the total amplitude of some arbitrary initial condition can **increase**!

In [ ]:
import matplotlib.pyplot as plt
from mpi4py import MPI
from eigentools import Eigenproblem, CriticalFinder
import dedalus.public as de
import numpy as np

First, we'll define the Orr-Sommerfeld equation in the usual way (following [Orszag 1971](https://www.cambridge.org/core/journals/journal-of-fluid-mechanics/article/abs/accurate-solution-of-the-orrsommerfeld-stability-equation/39D4D85F9939CC4E2F4A7EF127DFB046)). Because of the first order form of dedalus, this becomes four first order equations in $z$. 

Then we create an `Eigenproblem` object. 

In [ ]:
z = de.Legendre('z', 128)
d = de.Domain([z],comm=MPI.COMM_SELF)

orr_somerfeld = de.EVP(d,['w','wz','wzz','wzzz'],'c')
orr_somerfeld.parameters['alpha'] = 1.
orr_somerfeld.parameters['Re'] = 10000.
orr_somerfeld.substitutions['umean']= '1 - z**2'
orr_somerfeld.substitutions['umeanzz']= '-2'

orr_somerfeld.add_equation('dz(wzzz) - 2*alpha**2*wzz + alpha**4*w -1j*alpha*Re*((umean-c)*(wzz - alpha**2*w) - umeanzz*w) = 0')
orr_somerfeld.add_equation('dz(w)-wz = 0')
orr_somerfeld.add_equation('dz(wz)-wzz = 0')
orr_somerfeld.add_equation('dz(wzz)-wzzz = 0')

orr_somerfeld.add_bc('left(w) = 0')
orr_somerfeld.add_bc('right(w) = 0')
orr_somerfeld.add_bc('left(wz) = 0')
orr_somerfeld.add_bc('right(wz) = 0')

# create an Eigenproblem object
EP = Eigenproblem(orr_somerfeld)

Calling the `solve` method on the `Eigenproblem` object will solve the eigenvalue problem with 128 modes and then solve it again with $3/2 \cdot 128 = 192$ modes. Using the rejection algorithm detailed in Chapter 7 of Boyd, `eigentools` creates `EP.evalues` which holds only the good eigenvalues: those that are the same (to some specified tolerence) between both solves. 

Of course, `EP.evalues_low` and `EP.evalues_high` store the low and high resolution eigenvalues, respectively, if you need them. 

The `sparse=False` option tells Dedalus to use the dense eigenvalue solver, which will retrieve all the eigenmodes (as many as there are modes in the problem). Of course, about half of them will be unusably inaccurate; the rejection methods in `eigentools` hides those from us.

In [ ]:
EP.solve(sparse=False)

`eigentools` also provides a simple method to calculate pseudospectra. In order to calculate pseudospectra for the differential-algebraic equation systems Dedalus typically provides, we use the algorithm developed by [Embree and Keeler (2017)](https://intranet.math.vt.edu/people/embree/M105501.pdf).

This uses the sparse eigenvalue solver to construct an invariant subspace to approximate the pseudospectrum. Typically, it requires a rather high number of modes, $k$. Here we set $k = 100$. After we construct the pseudospectrum, we need to evaluate it at points in the complex plane of which the spectrum is a subset. We do that by choosing a set of real and imaginary points, and feeding them to the `EP.calc_ps()` method:

In [ ]:
k = 100

psize = 100
real_points = np.linspace(0,1, psize)
imag_points = np.linspace(-1,0.1, psize)
EP.calc_ps(k, (real_points, imag_points))

This adds data attributes `EP.pseudospectrum`, `EP.ps_real`, `EP.ps_imag` to the `EP` object, storing the pseudospectrum contours, and the real and imaginary grid for conveninece. 

Traditionally, one plots pseudospectra as logarithmic contours over top of the spectrum. Here, the contours are from -1 to -8 from outside in. This means that a perturbation of amplitude $\epsilon \simeq 10^{-1}$ to $\epsilon \simeq 10^{-8}$ will cause an $\mathcal{O}(1)$ change in the eigenvalues within that contour. Importantly, transient growth is possible within all of the contours that cross $c_i > 0$ with amplification factors proportional to $1/\epsilon$. Here, we can clearly see why channel flow is turbulent even when the only unstable mode has such a feeble growth rate!

In [ ]:
plt.scatter(EP.evalues_low.real, EP.evalues_low.imag,color='red')
plt.contour(EP.ps_real,EP.ps_imag, np.log10(EP.pseudospectrum),levels=[-8,-7,-6,-5,-4,-3,-2,-1],colors='k')
plt.axis('square')

plt.xlim(0,1)
plt.ylim(-1,0.1)
plt.xlabel(r"$c_r$")
plt.ylabel(r"$c_i$")

## Primative Form

One of the great advantages of Dedalus is that we can study problems without having to manipulate them into special forms, as in the case of the Orr-Sommerfeld operator. After all, this is nothing more than a special form of the 2-D Navier-Stokes equation linearized around $U(z)$.

We should be able to enter the "primitive" form of the equations and get equivalent results. Here, we definte a new Dedalus `EVP`, and then feed it to `eigentools`.

In [ ]:
primative = de.EVP(d,['u','w','uz','wz', 'p'],'c')
primative.parameters['alpha'] = 1.
primative.parameters['Re'] = 10000.

primative.substitutions['umean'] = '1 - z**2'
primative.substitutions['umeanz'] = '-2*z'
primative.substitutions['dx(A)'] = '1j*alpha*A' 
primative.substitutions['dt(A)'] = '-1j*alpha*c*A'
primative.substitutions['Lap(A,Az)'] = 'dx(dx(A)) + dz(Az)'

primative.add_equation('dt(u) + umean*dx(u) + w*umeanz + dx(p) - Lap(u, uz)/Re = 0')
primative.add_equation('dt(w) + umean*dx(w) + dz(p) - Lap(w, wz)/Re = 0')
primative.add_equation('dx(u) + wz = 0')
primative.add_equation('uz - dz(u) = 0')
primative.add_equation('wz - dz(w) = 0')
primative.add_bc('left(w) = 0')
primative.add_bc('right(w) = 0')
primative.add_bc('left(u) = 0')
primative.add_bc('right(u) = 0')


prim_EP = Eigenproblem(primative)

Again, we solve the dense eigenproblem just to get the full spectrum.

In [ ]:
prim_EP.solve(sparse=False)

And we recalculate the pseudospectrum, covering the same part of the complex plane.

In [ ]:
prim_EP.calc_ps(k, (real_points, imag_points))

Now, let's plot both the Orr-Sommerfeld form and the primitive form.

In [ ]:
plt.scatter(prim_EP.evalues.real, prim_EP.evalues.imag,color='red')
plt.scatter(EP.evalues.real, EP.evalues.imag,color='blue',marker='x')


clevels = [-8,-7,-6,-5,-4,-3,-2,-1]
OS_CS = plt.contour(EP.ps_real,EP.ps_imag, np.log10(EP.pseudospectrum),levels=clevels,colors='blue')
P_CS = plt.contour(prim_EP.ps_real,prim_EP.ps_imag, np.log10(prim_EP.pseudospectrum),levels=clevels,colors='red')

lines = [OS_CS.collections[0],P_CS.collections[0]]
labels = ['O-S form', 'primitive form']

plt.axis('square')
plt.xlim(0,1)
plt.ylim(-1,0.1)
plt.axhline(0,color='k',alpha=0.2)
plt.xlabel(r"$c_r$")
plt.ylabel(r"$c_i$")
plt.legend(lines, labels)
plt.tight_layout()
plt.savefig("OS_pseudospectra.png", dpi=300)

We see that the on the whole, there is very good agreement. The primitive form has more good eigenvalues (red dots with no corresponding blue crosses; none of the converse). Furthermore, the pseudospectra contours for the primitive form are slightly more accurate.